In [1]:
from __future__ import division                                         
import tensorflow as tf                                                 
tf.set_random_seed(1) 

import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad, Adam
from keras.utils import np_utils, generic_utils, to_categorical
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
import sys
import pickle

print ("Import Successful")

Import Successful


Using TensorFlow backend.


If want to train for 2 second utterances use '13L_2sec_mel_spectrograms.pickle' otherwise use '13L_mel_spectrograms.pickle'

In [2]:
with open('13L_mel_spectrograms.pickle','rb') as f:
    corpus,targets = pickle.load(f,encoding='latin1')


xtrain, xtest, ytrain, ytest = train_test_split(corpus, targets, test_size=0.2, random_state=42)
print('Training data dimensions : {}'.format(xtrain.shape))
print('Test data : {}'.format(xtest.shape))

Training data dimensions : (6531, 128, 501, 1)
Test data : (1633, 128, 501, 1)


In [3]:
# create model
model = Sequential()
    
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=xtrain[0].shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
    
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(8, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512,activation='relu'))

model.add(Dense(13, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])

# Train the model
model.fit(xtrain, to_categorical(ytrain),batch_size=128,shuffle=True,epochs=50,verbose=1,callbacks=[EarlyStopping(min_delta=0.001, patience=3)])


Epoch 1/50
6531/6531 [==============================] - 12s 2ms/step - loss: 2.4454 - acc: 0.1876
Epoch 2/50
 128/6531 [..............................] - ETA: 6s - loss: 2.2400 - acc: 0.2812

/home/priyam.jain/tensorflow-gpu-python3/lib/python3.5/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


6531/6531 [==============================] - 8s 1ms/step - loss: 2.1257 - acc: 0.3260
Epoch 3/50
6531/6531 [==============================] - 8s 1ms/step - loss: 1.8795 - acc: 0.3953
Epoch 4/50
6531/6531 [==============================] - 8s 1ms/step - loss: 1.7139 - acc: 0.4488
Epoch 5/50
6531/6531 [==============================] - 8s 1ms/step - loss: 1.5729 - acc: 0.5082
Epoch 6/50
6531/6531 [==============================] - 9s 1ms/step - loss: 1.4479 - acc: 0.5639
Epoch 7/50
6531/6531 [==============================] - 9s 1ms/step - loss: 1.2876 - acc: 0.6034
Epoch 8/50
6531/6531 [==============================] - 9s 1ms/step - loss: 1.1631 - acc: 0.6333
Epoch 9/50
6531/6531 [==============================] - 9s 1ms/step - loss: 1.0812 - acc: 0.6566
Epoch 10/50
6531/6531 [==============================] - 9s 1ms/step - loss: 0.9612 - acc: 0.6965
Epoch 11/50
6531/6531 [==============================] - 9s 1ms/step - loss: 0.8583 - acc: 0.7397
Epoch 12/50
6531/6531 [================

In [4]:
model.save('13LID_CNN.h5')

In [5]:
# Evaluate the model
scores = model.evaluate(xtest, to_categorical(ytest))
print('Loss: {}'.format(scores[0]))
print('Accuracy: {}'.format(scores[1]))

1633/1633 [==============================] - 1s 807us/step
Loss: 2.8141625142871276
Accuracy: 0.6436007348438457


In [6]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 499, 16)      160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 249, 16)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 249, 16)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 247, 16)       2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 123, 16)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 123, 16)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 121, 8)        1160      
__________

In [7]:
# Confusion matrix
from sklearn.metrics import confusion_matrix

ypred = model.predict(xtest)

ypred_enc = np.argmax(ypred, axis=1)
print('Predicted Labels dimensions : {}'.format(ypred_enc.shape))
print(confusion_matrix(ytest,ypred_enc))

Predicted Labels dimensions : (1633,)
[[124   0   0   0   6   2   1   0   3   0   0   3   1]
 [  1  82   3  13   1   7   1   6   3   4   1   2   2]
 [  2   1  79   1   8  10   1   3   7   4   9   9   4]
 [  0   0   0 107   0   1   0   0   3   3   1   0   1]
 [  9   1   8   0  82  10   3   4   4   1   4   4   0]
 [  0   3   0   0   2  93   1   0   9   3   0   0   0]
 [  5   3   3   3   4   0  87   2   4   9   0   4   0]
 [  5  13   2   8   5   0   2  90   3   2   0   0   0]
 [  3   3   8   7   1   7   4   4  62  22   2   3   1]
 [  4   4   5   8   4   5   7   6  42  36   3   3   3]
 [  0   1  14   1   5   4   1   0   4   0  78  15   7]
 [  2   0  11   5   2   3   4   1   3   2  24  54   2]
 [  0   1   4   4   5   4   1   0   6   4  11   1  77]]


In [8]:
import seaborn as sns

class_names = []
data_folder='13_language_dataset'
for lang in os.listdir(data_folder):
    if not lang.startswith('.'):
        class_names.append(lang)

class_names = np.asarray(class_names)

fig = sns.heatmap(confusion_matrix(ytest,ypred_enc),xticklabels=class_names,yticklabels=class_names)

print(fig)

AxesSubplot(0.125,0.11;0.62x0.77)


In [9]:
fig.get_figure().savefig('5sec_heatmap.png')